# Working with images - part II

### Region Of Interest (image) - ROI the numpy way

Sometimes, we will have to play with certain region of an Images (e.g. isolation of face after face detection)

Look at this image

![title](images/smiley.jpg)

In [1]:
import cv2
import numpy as np
img = cv2.imread('images/smiley.jpg')
cv2.imshow('smiley', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Now lets say we want select only the area where the evil smily (second from the left bottom line), for that we'll need to identify the region rectangle of that smiley, for that we can use many utilities programs (even the one showd above can help)

In our case:
* the upper left  side coordinates are (150, 200)
* the lower right side coordinates are (300, 350)  

So defining this ROI goes like this:  

img[y0:y1, x0:x1]

In [5]:
bad_smiley = img[200:350, 150:300]

![title](images/ROI_diagram.gif)

Now we can assign a value to the entire ROI either an array with the exact dimentions or we can simply give all pixels a value

In [25]:
import cv2
import numpy as np
img = cv2.imread('images/smiley.jpg')
img[200:350, 150:300] = img[0,0]
cv2.imshow('smiley', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### outcome:

![title](images/smiley_minus_roi.jpg)

### Image Addition (arithmetic)

You can add two images by OpenCV function, cv2.add() or simply by numpy operation, res = img1 + img2. <B>Both images should be of same depth and type, or second image can just be a scalar value.</B>

consider the folowing images:

![title](images/samurai.jpeg) +  ![title](images/china.jpeg)

In [27]:
import numpy as np
import cv2

im1 = cv2.imread('images/samurai.jpeg')
im2 = cv2.imread('images/china.jpeg')
im3 = im1 + im2
cv2.imshow('im1',im1)
cv2.imshow('im2',im2)
cv2.imshow('im3',im3)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Question: what happens when the sum of 2 pixels exceeds 255 ?

In [14]:
im1[200][200]

array([117, 187, 224], dtype=uint8)

In [15]:
im2[200][200]

array([  8,  92, 186], dtype=uint8)

In [16]:
im3[200][200]

array([125,  23, 154], dtype=uint8)

### Outcome of Samurai + chaina: 

![title](images/samurai_plus_china.jpeg)

### Image Blending (the cv2 way) 
This is also image addition, but different weights are given to images so that it gives a feeling of blending or transparency. Images are added as per the equation below:

g(x) = (1 - alpha)* f0(x) + alpha * f1(x)

By varying alpha from 0 -> 1, you can perform a cool transition between one image to another.

Here I took the two images shown above and blended them together. First image is given a weight of 0.7 and second image is given 0.3. cv2.addWeighted() applies following equation on the image.

dst = alpha*img1 + beta*img2 + gamma

Here gamma is taken as zero.

In [28]:
import numpy as np
import cv2

im1 = cv2.imread('images/samurai.jpeg')
im2 = cv2.imread('images/china.jpeg')
dst = cv2.addWeighted(im1, 0.7, im2, 0.3, 0)
cv2.imshow('im1',im1)
cv2.imshow('im2',im2)
cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### outcome:

![title](images/samurai_china_addWeighted.jpeg)

### Bitwise Operations (or manipulating a none rectangular area in an image)

This includes bitwise AND, OR, NOT and XOR operations. They will be highly useful while extracting any part of the image (as we will see in coming chapters), defining and working with non-rectangular ROI etc. Below we will see an example on how to change a particular region of an image.

I want to put OpenCV logo above an image. If I add two images, it will change color. If I blend it, I get an transparent effect. But I want it to be opaque. If it was a rectangular region, I could use ROI as we did in last chapter. But OpenCV logo is a not a rectangular shape. So you can do it with bitwise operations as below:

In [22]:
import cv2
import numpy as np

#loading 2 images
img1 = cv2.imread('images/samurai.jpeg')
cv2.imshow('1 orig image', img1)

img2 = cv2.imread('images/opencv-logo.png')
cv2.imshow('2 orig logo', img2)

#find the logo image dimentions
rows, cols, channels = img2.shape

#defining region of interest at the size of the logo 
#on the upper left side of img1 
roi = img1[0:rows, 0:cols ]

#converting the logo to grayscale
img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
cv2.imshow('3 logo converted to grayscale', img2gray)

#creating a mask with the threshold function
ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
cv2.imshow('4 mask', mask)

#now creating the inverted mask
mask_inv = cv2.bitwise_not(mask)
cv2.imshow('5 mask inverted', mask_inv)


#now black-out the area of logo in ROI
roi_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
cv2.imshow('6 roi bckground', roi_bg)


#take only region of logo from logo image.
roi_fg = cv2.bitwise_and(img2, img2, mask=mask)
cv2.imshow('7 roi foregroung ', roi_fg)

#summing up roi background with roi forgroung
dst = cv2.add(roi_bg, roi_fg)
cv2.imshow('8 adding the roi gg to fg', dst)

#replacing the corresponding area in the original image 
img1[0:rows, 0:cols ] = dst
cv2.imshow('9 result', img1)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Exercise
Create a slide show of images in a folder with smooth transition between images using cv2.addWeighted function